In [2]:
!pip install confluent_kafka

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 38.6 MB/s eta 0:00:00a 0:00:01


In [5]:
import pandas as pd 
import os 
import json
from confluent_kafka import Producer

In [6]:
!pip show confluent_kafka

Name: confluent-kafka
Version: 2.11.0
Summary: Confluent's Python client for Apache Kafka
Home-page: https://github.com/confluentinc/confluent-kafka-python
Author: 
Author-email: "Confluent Inc." <support@confluent.io>
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: 
Required-by: 


In [7]:
conf = {
    "bootstrap.servers" : "pkc-921jm.us-east-2.aws.confluent.cloud:9092",
    "security.protocol" : "SASL_SSL",
    "sasl.mechanisms" : "PLAIN",
    "sasl.username" : "SXPAZ35UGMGVACIZ",
    "sasl.password" :"Kms3Q3p6ej8YwSKseb4RY6gP/RBze+LuB7wPQvAW7sCd8vuzhU0pexn7MijKTAk9",
    "client.id" : "json-serial-producer"
}


producer = Producer(conf)

topic = "raw_topic"



%6|1753841750.533|GETSUBSCRIPTIONS|json-serial-producer#producer-1| [thrd:main]: Telemetry client instance id changed from AAAAAAAAAAAAAAAAAAAAAA to cTHxdIdpRPi6ysCGC0kCpw


In [ ]:
def delivery_report(err, msg):
    if err:
        print(f"Message Dilevery Failed : {err}")
    else:
        print(f"Message Delivery Sucessfully : {msg.key()} ")

def read_checkpoint(checkpoint_file):
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as file:
            return int(file.read().strip())
    return 0

def write_checkpoint(checkpoint_file,index):
    with open(checkpoint_file, 'w') as file:
        file.write(str(index))
    print(f"Checkpoint updated to line: {index}")


def handle_date(obj):
    if isinstance(obj, pd.Timestamp):
        return obj.strftime('%Y-%m-%d %H:%M:%S')
    raise TypeError(f"Object of type {type(obj).__name__} is not JSON serializable")
    
def stream_json_serially(file_path,checkpoint_file='/kaggle/working/checkpoint.txt'):
    last_sent_index = read_checkpoint(checkpoint_file)
    
    with open(file_path,'r') as file:
        for idx,line in enumerate(file):
            if idx < last_sent_index:
                continue
                
            try:
                record = json.loads(line)
                producer.produce(
                    topic,
                    key=str(record['review_id']),
                    value=json.dumps(record,default=handle_date).encode('utf-8'),
                    callback=delivery_report
                )
                
                producer.flush()
                
                write_checkpoint(checkpoint_file, idx + 1)
                
            except json.JSONDecodeError as e:
                print(f"Failed to decode JSON: {e}")
                
if __name__ == "__main__":
    stream_json_serially('/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json')

Message Delivery Sucessfully : b'9u1R2oilBgPXIPX6e2xCtg' 
Checkpoint updated to line: 7584
Message Delivery Sucessfully : b'W7WbM7QDNjRRlxQii9rUSQ' 
Checkpoint updated to line: 7585
Message Delivery Sucessfully : b'jgqWmC9KO14fdetHTfOdxw' 
Checkpoint updated to line: 7586
Message Delivery Sucessfully : b'SbiWIybUhK3-AnyWbO1mQg' 
Checkpoint updated to line: 7587
Message Delivery Sucessfully : b'XQLxqZPXkSJX4udMpCeLIw' 
Checkpoint updated to line: 7588
Message Delivery Sucessfully : b'gw3Unrf2IJx7riN3sFvGfw' 
Checkpoint updated to line: 7589
Message Delivery Sucessfully : b'NCnWeu2dEHG-2QL27gRx-g' 
Checkpoint updated to line: 7590
Message Delivery Sucessfully : b'NyQ3GkE0zOtnSwAorAmmdw' 
Checkpoint updated to line: 7591
Message Delivery Sucessfully : b'E1AQpH3R803jOiuAvqm0vg' 
Checkpoint updated to line: 7592
Message Delivery Sucessfully : b'zJUxQgDCQpZXVRfi0aCIRw' 
Checkpoint updated to line: 7593
Message Delivery Sucessfully : b'OEAPqysbldUGYnk8RPBdYA' 
Checkpoint updated to line: 7594